# Building A Multi Layer Perceptron

## Building Training Dataset

In [7]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plot
%matplotlib inline

In [2]:
words = open('../data/names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [14]:
# build vocab and mappings
chars = sorted(list(set(''.join(words))))
chtoi ={ch:i+1 for i, ch in enumerate(chars)} # char to int mapping
chtoi['.'] = 0 # add separator
itoch = {i:ch for ch, i in chtoi.items()} # int to char mapping
print(itoch)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [15]:
# building the dataset
block_size = 3 

`block_size` represents the context length i.e. how many characters does the model take to predict the next one.

In [17]:
X, Y = [], []

for w in words[:3]:
    print(w)
    context = [0] * block_size ## start with padded context of 0s
    for ch in w + '.':
        ix = chtoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itoch[i] for i in context), '->', itoch[ix])
        context = context[1:] + [ix] # update the context window

emma
... -> e
..e -> m
.em -> m
emm -> a
mma -> .
olivia
... -> o
..o -> l
.ol -> i
oli -> v
liv -> i
ivi -> a
via -> .
ava
... -> a
..a -> v
.av -> a
ava -> .


In [20]:
print(f'inputs:\n{X}\n')
print(f'targets:\n{Y}\n')

inputs:
[[0, 0, 0], [0, 0, 5], [0, 5, 13], [5, 13, 13], [13, 13, 1], [0, 0, 0], [0, 0, 15], [0, 15, 12], [15, 12, 9], [12, 9, 22], [9, 22, 9], [22, 9, 1], [0, 0, 0], [0, 0, 1], [0, 1, 22], [1, 22, 1]]

targets:
[5, 13, 13, 1, 0, 15, 12, 9, 22, 9, 1, 0, 1, 22, 1, 0]



In [21]:
X = torch.tensor(X)
Y = torch.tensor(Y)

Notice that out of the word "emma" we can generate 5 different example for inputs and targets.

In [22]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([16, 3]), torch.int64, torch.Size([16]), torch.int64)

## Building The Embedding Table

In [24]:
C = torch.randn((27, 2)) # creating a 2D vector space for a 27 token vocab size
C 

tensor([[-2.5095e-01,  6.3268e-01],
        [ 6.7109e-01, -6.0272e-01],
        [ 1.6476e-01, -1.6241e+00],
        [-4.4414e-01, -2.6809e-01],
        [ 1.9324e+00,  1.2073e+00],
        [-9.7633e-01, -1.6983e+00],
        [-5.4336e-01,  8.6897e-01],
        [ 2.1105e-03,  4.4206e-01],
        [-3.1510e-01, -1.7038e+00],
        [ 8.8760e-01, -1.7088e+00],
        [ 1.1976e+00,  4.5825e-01],
        [ 3.7455e-01,  3.0131e-01],
        [-1.2639e-01, -3.1526e-01],
        [-7.3973e-01,  1.1733e+00],
        [-2.5549e-01,  2.6838e-01],
        [-5.1717e-01, -9.5439e-01],
        [ 2.3148e+00, -6.8375e-01],
        [-9.4651e-01,  1.0856e+00],
        [-3.5373e-01,  1.7227e+00],
        [-1.2434e+00, -5.8994e-01],
        [ 2.1959e-01,  1.4281e+00],
        [-7.1352e-01, -1.0851e-01],
        [ 1.4134e+00,  9.6002e-01],
        [-2.3733e+00, -4.9556e-01],
        [-1.1405e-01, -1.9218e+00],
        [ 1.3865e+00,  1.0302e+00],
        [-5.4251e-01, -3.1156e+00]])

Embedding a single integer in the embedding lookup table

In [25]:
C[5] # row corresponsding to index 5 in the embedding table

tensor([-0.9763, -1.6983])

In [26]:
F.one_hot(torch.tensor(5), num_classes=27) # encoding integer 5 to a vector

tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0])

In [28]:
F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([-0.9763, -1.6983])

Notice that the output is identical. Because of all the 0s in the one-hot vector, all other values in C get masked out except for the row corresponding to the integer i.e. index 5. 

So the embedding of an integer can either be seen as the integer indexing into a lookup table C or it can be seen as the first layer of a larger NN. The neurons in this layer have no non-linearity and their weight matrix is C. We are encoding ints in one-hot and then feeding them to this layer.

Getting embeddings of multiple integers

In [29]:
C[[5, 6, 7]]

tensor([[-0.9763, -1.6983],
        [-0.5434,  0.8690],
        [ 0.0021,  0.4421]])

In [31]:
C[torch.tensor([5, 6, 7])]

tensor([[-0.9763, -1.6983],
        [-0.5434,  0.8690],
        [ 0.0021,  0.4421]])

In [33]:
# we can also index with multi dim tensors
C[X][:3]

tensor([[[-0.2509,  0.6327],
         [-0.2509,  0.6327],
         [-0.2509,  0.6327]],

        [[-0.2509,  0.6327],
         [-0.2509,  0.6327],
         [-0.9763, -1.6983]],

        [[-0.2509,  0.6327],
         [-0.9763, -1.6983],
         [-0.7397,  1.1733]]])

In [36]:
X.shape

torch.Size([16, 3])

In [37]:
C[X].shape # for everyone of the vectors in X we have a 2D embedding

torch.Size([16, 3, 2])

In [40]:
# example 
print(X[13, 2])
print(C[X][13, 2])
print(C[1])

tensor(1)
tensor([ 0.6711, -0.6027])
tensor([ 0.6711, -0.6027])


In [41]:
# so embedding all ints
emb = C[X]
emb.shape

torch.Size([16, 3, 2])